### Interactive Layer Clipping

## Step 1: Define your AOI

Draw a rectangle on the map using the rectangle control. This will be used as the bounding box of the clipped raster.

## Step 2: Define your target SRS

Provide your target SRS as an EPSG code.  The identified name of the EPSG code is provided for your reference.

You must also provide a cell size in your target coordinate system's units.

## Step 3: Click the `submit` button to start clipping

Logging will be printed below the map.

In [ ]:
import os
import logging
import sys
from urllib.parse import parse_qs

import ipywidgets
from IPython import get_ipython
import IPython.display

LOGGER = logging.getLogger(__name__)

# Check the query parameters if we're in voila
if (os.environ.get('SERVER_SOFTWARE', False) and 
        os.environ['SERVER_SOFTWARE'].startswith('voila')):      
    # localhost:8866?foo=bar,baz will return {'foo': ['bar', 'baz']}
    query_string = os.environ['QUERY_STRING']
    URL = os.environ['VOILA_SERVER_URL']
    parameters = parse_qs(query_string)
    display("query string parameters:", parameters)
else:
    # We're not running in voila, default to no parameters.
    URL = './'
    parameters = {}

In [ ]:
from ipyleaflet import basemap_to_tiles, basemaps
import ipywidgets
import time
import uuid

import dataportal_widgets
import output_options
import interactive_options

output_handler = output_options.OutputWidgetHandler()
stderr_handler = logging.StreamHandler(stream=sys.stderr)
logging.getLogger().addHandler(output_handler)
logging.getLogger().addHandler(stderr_handler)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('pygeoprocessing').setLevel(logging.DEBUG)

chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

if 'initial_layer' in parameters:
    initial_layer = parameters['initial_layer']
else:
    # just assume SRTM if the user hasn't specified a layer.
    initial_layer = "srtm-v3-1s"

m = dataportal_widgets.DataportalMap(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
    initial_layer=initial_layer
)

epsg_input = output_options.EPSG_INPUT
epsg_label = output_options.EPSG_LABEL
pixel_sizes_label = output_options.PIXEL_SIZES_LABEL


def _detect_srs_name(change):
    try:
        if isinstance(change['new'], str):
            epsg_code = int(change['new'])
        elif isinstance(change['new'], dict):
            epsg_code = int(change['new']['value'])
        else:
            raise Exception(change)
    except (ValueError, TypeError):
        epsg_code = None
    except KeyError:
        # no new value was set; skip
        return
        
    label, pixel_size_text = output_options.parse_epsg_code(epsg_code)
        
    epsg_label.value = label
    pixel_sizes_label.value = pixel_size_text
    
epsg_input.observe(_detect_srs_name, 'value')
epsg_input.disabled = True

pixel_size_x_input = output_options.PIXEL_SIZE_X_INPUT
pixel_size_x_input.disabled = True
pixel_size_y_input = output_options.PIXEL_SIZE_Y_INPUT 
pixel_size_y_input.disabled = True

reproject_cb = ipywidgets.Checkbox(
    value=False,
    description='Reproject to a local projection',
    layout={'width': 'max-content'},
    disabled=False,
    indent=False
)

def _enable_projection_info(value):
    epsg_input.disabled = not value['new']
    pixel_size_x_input.disabled = not value['new']
    pixel_size_y_input.disabled = not value['new']

reproject_cb.observe(_enable_projection_info, 'value')

dataset_label = ipywidgets.Label("")
pixel_size_label = ipywidgets.Label(value="")
input_descriptions_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Source layer</b>"),
    ipywidgets.HBox([ipywidgets.Label(value="Name:"), dataset_label]),
    ipywidgets.HBox([ipywidgets.Label(value="Projection: EPSG:4326")]),
    ipywidgets.HBox([ipywidgets.Label(value="Pixel size:"), pixel_size_label])
])
def _update_source_layer_labels(value):
    new_ds_key = value['new']
    dataset_label.value = dataportal_widgets.DATASET_DIRECTORY[new_ds_key]['title']
    try:
        pixel_size_label.value = str(dataportal_widgets.DATASET_DIRECTORY[new_ds_key]['data']['pixel_size'])
    except KeyError:
        pixel_size_label.value = 'unavailable'
    

m.layers_dropdown.observe(_update_source_layer_labels, 'value')
_update_source_layer_labels({'new': initial_layer})  # prepopulate with initial layer

output_params_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Output options</b>"),
    reproject_cb,
    ipywidgets.HBox([epsg_input, epsg_label]),
    ipywidgets.HBox(
        [pixel_size_x_input, pixel_size_y_input, pixel_sizes_label]),
])

logging_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Logging</b>"),
    output_handler.out,
])

submit = ipywidgets.Button(
    description='Clip raster',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clip the selected raster by the selected bounding box',
    icon='crop' # (FontAwesome names without the `fa-` prefix)
)
download_link = ipywidgets.HTML()
submit_hbox = ipywidgets.HBox([submit, download_link])
def _submit(button):
    button.disabled=True
    try:
        dataset_key = m.selected_map()
        timestamp = time.time()
        target_path = f'/mnt/gcs/jupyter-app-temp-storage/{dataset_key}-{timestamp}-{uuid.uuid4()}.tif'
        source_raster_path = f"/vsicurl/{dataportal_widgets.get_url(m.selected_map())}"
        if reproject_cb.value:
            interactive_options.compute(
                source_raster_path,
                m.selected_bbox(),
                epsg_input.value,
                target_path,
                (float(pixel_size_x_input.value), float(pixel_size_y_input.value)),
            )
        else:
            interactive_options.compute(
                source_raster_path,
                m.selected_bbox(),
                4326,  # assumes all rasters are already EPSG:4326
                target_path,
                None  # use existing pixel size
            )
        download_url = target_path.replace('/mnt/gcs', 'https://storage.googleapis.com')
        LOGGER.info(f"File can be downloaded from {download_url}")
        download_link.value = f"<a href={download_url}>Download {os.path.basename(download_url)}</a>"
    except Exception:
        LOGGER.exception("Compute/upload failed")
    finally:
        # always re-enable the button afterwards
        button.disabled=False
    
submit.on_click(_submit)

display(m)
display(input_descriptions_box)
display(output_params_box)
display(submit_hbox)
display(logging_box)
